In [7]:
# Colab only:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [0]:
# Colab only:
!pip3 install -q gensim

In [0]:
import tensorflow as tf
import itertools
import numpy as np
from scipy import spatial
from scipy.stats import norm
import nltk.data
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import reuters
from nltk. corpus import gutenberg
from nltk.corpus import brown
from nltk.tokenize import sent_tokenize
from gensim.models import KeyedVectors
from keras.layers import Input, Dense, Lambda, Layer, LSTM, RepeatVector, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras import backend as K
from keras import metrics, objectives
import nltk




In [10]:
# Colab only:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True


### Открываем исходники,склеенные при помощи строки "@...". Разделяем и токенизируем исходники при помощи nltk. Представляем коды в виде массивов последовательностей токенов


In [19]:
file_content = open("./data/sources.txt").read()
file_content += "GOODEXAMPLESTART" + " | " + "BADEXAMPLESTART" + " | " +  "EXAMPLESTOP" 
file_content_arr = file_content.split("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

tokenized = []
for fa in file_content_arr:
   tokenized.append(nltk.word_tokenize(fa))

print("Tokens examples: ", tokenized[0][:20])

Tokens examples:  ['/*', 'PR', 'preprocessor/41445', '*/', '/*', '{', 'dg-do', 'compile', '}', '*/', '/*', '{', 'dg-options', '``', '-gdwarf', '-O0', '-dA', '-fno-merge-debug-strings', "''", '}']


### Растягиваем в 100мерном пространстве последовательности токенов при помощи Word2Vec. Так же добавляем в модель w2v отдельностоящие старт/стоп слова для генирации послдовательностей GOODEXAMPLESTART/ BADEXAMPLESTART/EXAMPLESTOP 

In [20]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

# w2v_model = Word2Vec.load('./models/w2v_cpp_model.h5')


# W2V model train:
w2v_model = Word2Vec(tokenized, size=100, window=5, min_count=1, workers=4)
w2v_model.train(tokenized, total_examples=len(tokenized), epochs=10)
w2v_model.wv.syn0[w2v_model.wv.vocab["GOODEXAMPLESTART"].index] = np.ones((1,100)).flatten()
w2v_model.wv.syn0[w2v_model.wv.vocab["EXAMPLESTOP"].index] = np.zeros((1,100)).flatten()
tmp = np.zeros((1,100)).flatten()
tmp[::2]=1
w2v_model.wv.syn0[w2v_model.wv.vocab["BADEXAMPLESTART"].index] = tmp
# w2v_model.save('./models/w2v_cpp_model.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [21]:
w2v_model.wv.similar_by_word('i')

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:1366: RuntimeWarning: invalid value encountered in true_divide
  self.vectors_norm = (self.vectors / sqrt((self.vectors ** 2).sum(-1))[..., newaxis]).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('j', 0.7125623822212219),
 (';', 0.6415524482727051),
 ('0', 0.6092534065246582),
 ('10', 0.5986620783805847),
 ('16', 0.5569596290588379),
 ('k', 0.5469374656677246),
 ('dst', 0.5397963523864746),
 ('tot', 0.5363589525222778),
 ('=', 0.5350142121315002),
 ('sum1', 0.5293612480163574)]

### Урезаем количество последовательностей для теста работоспособности модели в Colab

In [0]:
# We are cutting tokenized array due to memory limit
tokenized = tokenized[:100]

### Векторизуем последовательности токенов, чтобы вместо последовательностей слов/символов кода получить последовательности векторов из w2v

In [24]:
# Cooking sequences for seq2seq

# Encoder sequence to w2v seq
us_encoder_input_data = []

for seq in tokenized:
  us_encoder_input_data.append([])
  for word in seq:
    us_encoder_input_data[-1].append(w2v_model.wv.word_vec(word))
  us_encoder_input_data[-1] = np.array(us_encoder_input_data[-1])
  
  
# Decoder sequence to w2v seq
start_word = "GOODEXAMPLESTART"
start_word_vec = w2v_model[start_word]
print('start_word_vec', start_word_vec.shape)

stop_word = "EXAMPLESTOP"
stop_word_vec = w2v_model[stop_word]
print('stop_word_vec', stop_word_vec.shape)

is_first_iteration = True
us_decoder_input_data = []
us_decoder_target_data = []

for seq in tokenized:
    us_decoder_input_data.append([])
    us_decoder_target_data.append([])
    #   start word appends only to decoder_input_data
    us_decoder_input_data[-1].append(start_word_vec)
    for word in seq:
        word_vectorized = w2v_model.wv.word_vec(word)
        us_decoder_input_data[-1].append(word_vectorized)
        us_decoder_target_data[-1].append(word_vectorized)
    #   stop word appends only to decoder_target_data
    us_decoder_target_data[-1].append(stop_word_vec)
    
    us_decoder_input_data[-1] = np.array(us_decoder_input_data[-1])
    us_decoder_target_data[-1] = np.array(us_decoder_target_data[-1])
    

start_word_vec (100,)
stop_word_vec (100,)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


### Создаем квадратные матрицы (по максимальным длинам наших массивов последовательностей). Данные матрицы удобны и пригодны для подачи на вход нейросети

In [27]:
max_sentence_length=0
for a in us_decoder_input_data:
  if max_sentence_length < a.shape[0]:
    max_sentence_length = a.shape[0]
print(max_sentence_length)

encoder_input_data = np.zeros((
    len(us_encoder_input_data),
    max_sentence_length,
    100,
))
decoder_input_data = np.zeros(encoder_input_data.shape)
decoder_target_data = np.zeros(encoder_input_data.shape)

328


### Копируем векторизованные последовательности в матрицы

In [29]:
# matricize sequences
for i, seq in enumerate(us_encoder_input_data):
  for j, word in enumerate(seq):
    for h in range(100):
      encoder_input_data[i,j,h] = us_encoder_input_data[i][j,h]
      
      
for i, seq in enumerate(us_decoder_input_data):
  for j, word in enumerate(seq):
    for h in range(100):
      decoder_input_data[i,j,h] = us_decoder_input_data[i][j,h]
      decoder_target_data[i,j,h] = us_decoder_target_data[i][j,h]
      
print(encoder_input_data.shape, decoder_input_data.shape, decoder_target_data.shape)

(100, 328, 100) (100, 328, 100) (100, 328, 100)


### Задаем параметры нейросети

In [0]:
# Neural Network
epsilon_std=0.01
batch_size = 100  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = encoder_input_data.shape[0]  # Number of samples to train on.

### Создаем нейросеть, это обычная Seq2Seq модель нейросети, взятая из блога Keras (https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)
### Наша модель учится генерировать те же самые последовательности, которые были поданы ей на вход.

### Но при этом, на выходе Encoder-а мы заворачиваем его стейты (state_h, state_c) в нейросеть с Variational слоями. Эти слои взяти из примера Mnist VAE (https://github.com/keras-team/keras/blob/master/examples/variational_autoencoder.py)

### При помощи такого хода мы можем выучить распределение слоев  (state_h, state_c) и, в дальнейшем, беря из этого распределения сэмпл, и, пропуская его через декодер, мы можем получить на выходе из декодера рандомно сгенеренную последовательность из интересующего нас распределения


In [81]:
# Seq2Seq + VAE model
def vae_loss(x, x_decoded_mean):
    x = K.flatten(x)
    x_decoded_mean = K.flatten(x_decoded_mean)
    xent_loss = decoder_input_data.shape[2] * objectives.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + sd_h - K.square(mn_h) - K.exp(sd_h), axis=-1)
    kl_loss -= 0.5 * K.mean(1 + sd_c - K.square(mn_c) - K.exp(sd_c), axis=-1)
    return xent_loss + kl_loss
  
# Define an input sequence and process it.

# Encoder
encoder_inputs = Input(shape=(None, decoder_input_data.shape[2]))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.


"""
Original Variational layer from Mnist vae:
mn = tf.layers.dense(x, units=n_latent)
sd       = 0.5 * tf.layers.dense(x, units=n_latent)            
epsilon = tf.random_normal(tf.stack([tf.shape(x)[0], n_latent])) 
z  = mn + tf.multiply(epsilon, tf.exp(sd))

"""
# Learn distribution from Hidden state
# Layer were incapsulated into Lambda: 
mn_h = Dense(latent_dim)(state_h)
sd_h = Lambda(lambda x: tf.multiply(0.5, Dense(latent_dim)(x)))(state_h)
epsilon_h =  Lambda(lambda x: tf.random_normal(tf.stack([tf.shape(x)[0], latent_dim])))(state_h) 
z_h = Lambda(lambda args: tf.add(args[0], tf.multiply(args[1], tf.exp(args[2]))))([mn_h, epsilon_h, sd_h])

# Learn distribution from Cell state
# Layers were incapsulated with Lambda: 
mn_c = Dense(latent_dim)(state_c)
sd_c = Lambda(lambda x: tf.multiply(0.5, Dense(latent_dim)(x)))(state_c)
epsilon_c = Lambda(lambda x: tf.random_normal(tf.stack([tf.shape(x)[0], latent_dim])))(state_c) 
z_c  = Lambda(lambda args: tf.add(args[0], tf.multiply(args[1], tf.exp(args[2]))))([mn_c, epsilon_c, sd_c])

encoder_states = [z_h, z_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, decoder_input_data.shape[2]))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(decoder_input_data.shape[2], activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer='rmsprop', loss=vae_loss)
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Train on 80 samples, validate on 20 samples
Epoch 1/100
80/80 [==============================] - 9s 114ms/step - loss: 7.0034 - val_loss: -3.0129
Epoch 2/100


KeyboardInterrupt: ignored

### Описываем модель генератора рандомных последовательностей кода

In [0]:
# Define sampling models

encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

### Описываем функцию, которая циклически токен за токеном генерирует последовательность кода при помощи модели генератора.

### Основное отличие от обычной функции генерации из модели Seq2Seq в том, что мы подаем на вход декодеру стейты, взятые из выученного распределения (вместо стейтов, взятых на выходе энкодера)

In [0]:
def sampling(args):
    z_mean_, z_log_var_ = args
    batch_size = K.shape(z_mean_)[0]
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0., stddev=epsilon_std)
    return z_mean_ + K.exp(z_log_var_ / 2) * epsilon

def decode_sequence(input_seq):
    # Encode the input as state vectors.
#     states_value = encoder_model.predict(input_seq)
    states_value = [np.random.normal(0, 1, latent_dim).reshape((1, latent_dim)) for _ in range(2)]

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, 100))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = w2v_model.wv['GOODEXAMPLESTART']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = w2v_model.wv.similar_by_vector(output_tokens[0, -1, :])[0][0]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == 'EXAMPLESTOP' or
           len(decoded_sentence) > max_sentence_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, 100))
        target_seq[0, 0] = w2v_model.wv['GOODEXAMPLESTART']

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [86]:
for seq_index in range(2):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
#     print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


-
Decoded sentence: GOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTART
-
Decoded sentence: GOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTARTGOODEXAMPLESTART
